# Get Parser based statistics


In [ ]:
!pip -q install jedi==0.10
!pip -q install torch==1.12.1
!pip -q install -U pip setuptools wheel
!pip -q install -U spacy
!pip -q install wget
!pip -q install benepar
!pip -q install deep-translator

In [ ]:
import spacy
import benepar
import svgling
from pathlib import Path
import en_core_web_sm
import nltk
import pickle as pkl
from nltk.stem import WordNetLemmatizer
from deep_translator import GoogleTranslator

nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()
eng_dep = en_core_web_sm.load()

In [ ]:
def word_analyzer(word, tag):
    ar = ""
    tag2= tag
    if tag == "<malay>":
        tag2 = "</malay>"
    if tag == "(":
        tag2 = ")"
    if tag == "[":
        tag2 = "]"
    if tag == "<":
        tag2 = ">"
    if tag == "{":
        tag2 = "}"
    n = len(tag)
    k = len(tag2)
    if word.startswith(tag) and word.endswith(tag2):
            ar = word[n:-k]
    elif word.startswith(tag):
            ar = word[n:]
    elif word.endswith(tag2):
            ar = word[:-k]
    return ar

In [ ]:
def data_cleanser(string):
    words = string.split()
    eng_ar = []
    mal_ar = []
    flag = False
    flag2 = False
    eng = ""
    mal = ""
    ar = []
    sent = ""
    for i in range(len(words)):
        word = words[i]
        if i == 0 and word.startswith("<malay>"):
            flag = True
        if "_" in word:
            wrd = ""
            for j in range(len(word)):
                if word[j] != "_":
                    wrd += word[j]
            word = wrd
            if flag2:
                mal += word+ " "
            else:
                eng += word+ " "
        elif word.startswith("<malay>") or word.endswith("</malay>"):
            word2 = word
            if word.startswith("<malay>"):
                flag2 = True
                eng_ar.append(eng)
                eng = ""
            word = word_analyzer(word, "<malay>")
            mal += word+ " "
            if word2.endswith("</malay>"):
                flag2 = False
                mal_ar.append(mal)
                mal = ""
        elif word.startswith("(") or word.endswith(")"):
            word = word_analyzer(word, "(")
            if flag2:
                mal += word+ " "
            else:
                eng += word+ " "
        elif word.startswith("[") or word.endswith("]"):
            word = word_analyzer(word, "[")
            if flag2:
                mal += word+ " "
            else:
                eng += word+ " "
        elif word.startswith("<") or word.endswith(">"):
            word = word_analyzer(word, "<")
            if flag2:
                mal += word+ " "
            else:
                eng += word+ " "
        elif word.startswith("{") or word.endswith("}"):
            word = word_analyzer(word, "{")
            if flag2:
                mal += word+ " "
            else:
                eng += word+ " "
        elif word.startswith("!") or word.endswith("!"):
            word = word_analyzer(word, "!")
            if flag2:
                mal += word+ " "
            else:
                eng += word+ " "
        elif word.startswith("#") or word.endswith("#"):
            word = word_analyzer(word, "#")
            if flag2:
                mal += word+ " "
            else:
                eng += word+ " "
        elif word.endswith("~"):
            continue
        else:
            if flag2:
                mal += word+ " "
            else:
                eng += word+ " "
    if mal == "" and eng != "":
        eng_ar.append(eng)
    elif mal != "" and eng == "":
        mal_ar.append(mal)
    return eng_ar, mal_ar, flag

In [ ]:
def translate_array(mal_ar):
    eng_ar = []
    for data in mal_ar:
        eng_ar.append(GoogleTranslator(source='ms', target='en').translate(data[:-1]))
    return eng_ar

In [ ]:
def check_parser(txt, eng_ar, mal_eng_ar, flag):
    i = 0
    j = 0
    if flag:
        while i<len(eng_ar) or j<len(mal_eng_ar):
            if j<len(mal_eng_ar):
                if txt in mal_eng_ar[j] or txt in lemmatizer.lemmatize(mal_eng_ar[j]):
                    return True
                j += 1
            if i<len(eng_ar):
                if txt in eng_ar[i] or txt in lemmatizer.lemmatize(eng_ar[i]):
                    return False
                i += 1
        return False
    else:
        while i<len(eng_ar) or j<len(mal_eng_ar):
            if i<len(eng_ar):
                if txt in eng_ar[i] or txt in lemmatizer.lemmatize(eng_ar[i]):
                    return False
                i += 1
            if j<len(mal_eng_ar):
                if txt in mal_eng_ar[j] or txt in lemmatizer.lemmatize(mal_eng_ar[j]):
                    return True
                j += 1
        return False

In [ ]:
filename = "CS_Data.txt"
with open(filename) as f:
    cs_data = f.readlines()

filename2 = "Eng_CS_final_Data.txt" # Translated file of code-switched data using Deep-Translator
with open(filename2) as f2:
    eng_cs_data = f2.readlines()

In [ ]:
ar_eng_parser = dict()
ar_mal_parser = dict()
exc = []

for i in range(len(cs_data)):
    try:  
        cs_sent = cs_data[i]
        eng_cs_sent = eng_cs_data[i]
        eng_ar, mal_ar, flag = data_cleanser(cs_sent)
        mal_eng_ar = translate_array(mal_ar)
        sent = ""
        if flag:
            i = 0
            j = 0
            while i<len(mal_eng_ar) or j < len(eng_ar):
                if i<len(mal_eng_ar):
                    sent += mal_eng_ar[i] + " "
                    i += 1
                if j < len(eng_ar):
                    sent += eng_ar[j]
                    j += 1
        else:
            i = 0
            j = 0
            while i<len(mal_eng_ar) or j < len(eng_ar):
                if j < len(eng_ar):
                    sent += eng_ar[j]
                    j += 1
                if i<len(mal_eng_ar):
                    sent += mal_eng_ar[i] +" "
                    i += 1
        
        doc = eng_dep(eng_cs_sent)
        for token in doc:
            dep = token.dep_
            fg = check_parser(token.text, eng_ar, mal_eng_ar, flag)
            if fg:
                if dep not in ar_mal_parser.keys():
                    ar_mal_parser[dep] = 1
                else:
                    ar_mal_parser[dep] += 1
            else:
                if dep not in ar_eng_parser.keys():
                    ar_eng_parser[dep] = 1
                else:
                    ar_eng_parser[dep] += 1
    except:
        exc.append(i)

In [ ]:
list_of_parser_tags = ["ROOT", "acl", "acomp", "advcl", "advmod", "agent", "amod", "appos", "attr", "aux", "auxpass", "case", "cc", "ccomp", "compound", "conj", "csubj", "csubjpass", "dative", "dep", "det", "dobj", "expl", "intj", "mark", "meta", "neg", "nmod", "npadvmod", "nsubj", "nsubjpass", "nummod", "oprd", "parataxis", "pcomp", "pobj", "poss", "preconj", "predet", "prep", "prt", "punct", "quantmod", "relcl", "xcomp"]

In [ ]:
df = pd.DataFrame(column = ["Parser Tag", "# English Instance", "# Malay Instances", "Malay Percentage"])
for dep in list_of_parser_tags:
    try: 
        eng = ar_eng_parser[dep]
    except: 
        eng = 0
    try: 
        mal = ar_mal_parser[dep]
    except:
        mal = 0
    if eng+mal != 0:
        df.loc[len(df.index)] = [dep, eng, mal, round(mal*100/(eng+mal), 2)]

df.to_excel("Parser_results.xlsx", index = False)